# Capsule

核心思想是： 每个capsule代表一个特征。

具体的解释，请看 [揭开迷雾，来一顿美味的Capsule盛宴 By 苏剑林](https://kexue.fm/archives/4819)


In [1]:
import torch
import torchvision
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn

In [2]:
# 定义 squash 函数
def squash(x, p=2, dim=1, keepdim=True):
    """
        params: x (num*feature), p: 几范数, dim: 对哪个维度求范数, keepdim: 保持维度一致
        return: squash_x (b*m)
    """
    squash_norm = torch.norm(x, p, dim, keepdim)
    scale = torch.sqrt(squash_norm) / (1 + squash_norm)
    return scale * x

In [3]:
# 计算softmax
def getSoftmax(u, v, dim=1):
    """
        params: u: 下一层的特征(num1*feature), v: 上一层的特征(num2*feature)
    """
    b = torch.mm(v, u.t()) # (num2 * num1)
    return F.softmax(b, dim)

In [4]:
class CapsuleLayer(nn.Module):
    def __init__(self, in_capsule_num, out_capsule_num, routings):
        super(CapsuleLayer, self).__init__()
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.linear = nn.Linear(in_capsule_num, out_capsule_num, bias=False)
        
    def forward(self, u_vecs):
        
        
        
    

SyntaxError: invalid syntax (<ipython-input-4-c597969ba41f>, line 7)